Uses the outputs from the other notebooks to calculate an I/F spectrum.

In [ ]:
import numpy as np
from astropy.io import fits
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import math
from scipy import interpolate
from numpy import pi, r_
from scipy import optimize
# error
from uncertainties import ufloat
from uncertainties.umath import *  # sin(), etc.
#>>> x = ufloat(1, 0.1)  # x = 1+/-0.1

In [ ]:
wl_sci = np.linspace(470,950,241) # NAIC wavelengths

In [ ]:
# Jupiter information:

# Load F_obs (raw Jupiter spectrum, from Geo_cal)
F_obs_error = np.loadtxt('/Users/dahlek/Desktop/nemesis_inputs_march2019/NEB_averaged_geo5')

# pull airmasses
directory = '/Volumes/external_hd/march2017/redo/jupiter/'
imlist = np.loadtxt('/Volumes/external_hd/march2017/march_2017_sciimages',dtype=str)

wl_airmass = []
jup_airmass = []

#collect info on fringe-corrected spectrum
for i in range(0,len(imlist)):
    im = fits.open(directory+imlist[i])
    if im[0].header['rfon'] == 1:
        wl_airmass.append(im[0].header['lambda'])
        jup_airmass.append(im[0].header['airmass'])
    im.close()

plt.plot(wl_sci,F_obs_error[:,1])
plt.title('Raw Jupiter spectrum')
plt.ylabel('DN/sec')
plt.xlabel('Wavelength (nm)')
plt.show()

plt.plot(wl_sci,jup_airmass)
plt.title('Jupiter cube airmass')
plt.ylabel('Airmass')
plt.xlabel('Wavelength (nm)')
plt.show()

In [ ]:
# Other information:

# Load Beta (from Atm_fitting)
beta = np.loadtxt('/Users/dahlek/Desktop/beta_test_2020')

# Load optical depth (from Atm_fitting)
tau = np.loadtxt('/Volumes/external_hd/march2017/redo/tau_march2017_noVir85')

# Load Kurucz model spectra
F_S = np.loadtxt('/Users/dahlek/Desktop/Vega_and_solar_Kurucz_spectra')[:,2]

In [ ]:
plt.errorbar(wl_sci,beta[:,1])
plt.errorbar(wl_sci,beta[:,1],color='blue',ecolor='g',yerr=np.array(beta[:,2]))
plt.title('Beta')
plt.ylabel('(DN/sec)/(erg/sec/cm2/A)')
plt.show()

plt.plot(wl_sci,tau[:,1])
plt.errorbar(wl_sci,tau[:,1],color='blue',ecolor='g',yerr=np.array(tau[:,2]))
plt.title('Derived Optical Depth')
plt.show()

plt.plot(wl_sci,F_S)
plt.errorbar(wl_sci,F_S,color='blue',ecolor='g',yerr=np.array(F_S*0.01))
plt.title('Model Solar spectrum')
plt.ylabel('erg/sec/cm2/A')
plt.show()

In [ ]:
# Constants:

# Platescales (don't take proper motion into account):
#Old camera (pre-Jan 2018):
#0.10593070086707904+/-0.006012429373929471
#New camera (post-Jan 2018):
#0.0847900011797713+/-0.004432559233932486


def omega_maker(platescale):
    '''Convert platescale to radians and square'''
    return (platescale/206265.)**2

# Solid angle of a pixel on NAIC CCD (pre-2018 camera). 
# Derived via double star measurements.
omega_with_error = ufloat(2.4724062e-13,1.4033e-14) # steradians
#omega = #0.10593+/-0.006012429373929471 "/pix

r = 5.45 #distance to jupiter from Sun, AU in march 2017

In [ ]:
IF = []
IF_error = []

for i in range(0,len(wl_sci)):
    
    # define each value and its error at that wavelength
    beta_error = ufloat(beta[i,1],beta[i,2])
    F_obs = ufloat(F_obs_error[i,1],F_obs_error[i,8])
    tau_error = ufloat(tau[i,1],tau[i,2])
    chi = jup_airmass[i]
    F_Solar = ufloat(F_S[i],F_S[i]*0.01) # assuming 1% error
    
    top = math.pi*beta_error*F_obs*np.e**(tau_error*chi)    
    bottom = omega_with_error*(F_Solar/(r**2))
    
    g = top/bottom
    
    IF.append(g.nominal_value)
    IF_error.append(g.std_dev)

In [ ]:
plt.plot(wl_sci,IF)
plt.errorbar(wl_sci,IF,color='blue',ecolor='c',yerr=np.array(IF_error))
plt.title('I/F spectrum')
plt.ylabel('I/F')
plt.xlabel('Wavelength (nm)')
plt.show()